In [1]:
from transfer_learning_publication.data import CaravanDataSource
from transfer_learning_publication.transforms import PipelineBuilder, Log, ZScore
from transfer_learning_publication.containers import TimeSeriesCollection

In [2]:
caravan = CaravanDataSource(
    base_path="/Users/cooper/Desktop/LSH_hive_data",
    region="tajikkyrgyz"
)

basins = caravan.list_gauge_ids()

ts_data = caravan.get_timeseries(
    gauge_ids=basins[:2],
    columns=["streamflow", "total_precipitation_sum"],
)

static_data = caravan.get_static_attributes(
    gauge_ids=basins[:2],
    columns=["area", "high_prec_freq"],
)

In [ ]:
# static_collection = caravan.to_time_series_collection(ts_data)
# static_collection.feature_indices

ValueError: Gauge 'tajikkyrgyz_15013' has null values in columns: ['streamflow']. This indicates an upstream processing error.

In [ ]:
preprocessing_pipelines = (
    PipelineBuilder(group_identifier="gauge_id")
    .add_per_basin(Log(), columns=["streamflow"])
    .add_global(ZScore(), columns=["streamflow", "total_precipitation_sum"])
    .build()
)

data_transformed = preprocessing_pipelines.fit_transform(ts_data.collect())

data_inv = preprocessing_pipelines.inverse_transform(data_transformed)

In [ ]:
ts_data.collect().head(), data_transformed.head(), data_inv.head()

In [ ]:
caravan.write_timeseries(
    data_transformed,
    "/Users/cooper/Desktop/transfer-learning-publication/data",
    True
)